# Multi Greek Hedging

In [2]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np

from utils.process import process_options,process_underlying,process_rf
from utils.greeks import delta, gamma, vega, theta, rho
from utils.implied_volatility import implied_vol_b76
from tqdm import tqdm

tqdm.pandas()


In [3]:
option_data = pd.read_csv("../data/SPYOPT_processed.csv",parse_dates=True)
underlying = pd.read_csv("../data/SPYFUT_processed.csv",parse_dates=True).rename(columns={'close': 'futures_close'})
rf_rate = pd.read_csv("../data/RFRate.csv", parse_dates=True).rename(columns={'rate': 'rf_rate'})

In [4]:
all_data = pd.merge(option_data, 
                    underlying[['ts_event', 'expiration_date', 'futures_close']], 
                    on=['ts_event', 'expiration_date'], 
                    how='left')

all_data = pd.merge(all_data, 
                    rf_rate[['ts_event', 'rf_rate']], 
                    on='ts_event', 
                    how='left')

all_data['rf_rate'] = all_data['rf_rate'].ffill().bfill()
all_data['rf_rate'] = 12 *np.log(1 + all_data['rf_rate']/1200)

all_data['ts_event'] = pd.to_datetime(all_data['ts_event'])
all_data['expiration_date'] = pd.to_datetime(all_data['expiration_date'])

all_data = all_data.set_index('ts_event')

all_data['futures_close'] = all_data['futures_close'].ffill()

all_data = all_data.reset_index()

all_data = all_data.drop(columns=['open', 'high', 'low', 'volume'])

all_data = all_data.rename(columns={'close': 'Value','futures_close': 'S','rf_rate': 'r','strike': 'K'})


all_data['T'] = (all_data['expiration_date'] - all_data['ts_event']).dt.days / 365.0

# Preparing Data

Since we are going to just be dealing with calls, we can filter out all the puts that are being traded.

In [5]:
#Reindexing
def grouper1(group):
    start_date = group['ts_event'].min()
    end_date = group['ts_event'].max()
    group = group.set_index('ts_event').reindex(pd.date_range(start=start_date, end=end_date, freq='D')).reset_index()
    group['option_type'] = group['option_type'].ffill()
    group['K'] = group['K'].ffill()
    group['expiration_date'] = group['expiration_date'].ffill()
    return group

def filter1(group):
    if len(group) < 10:
        return False
    
    S0    = group['S'].iloc[0]                      # spot
    K     = group['K'].iloc[0]
    T    = group['T'].iloc[0]                      # time to maturity
    r = group['r'].iloc[0]                      # risk-free rate
    upper = S0 *np.exp(r*T + 1.64*0.2*np.sqrt(T))  # upper bound
    lower = S0 *np.exp(r*T - 1.64*0.2*np.sqrt(T))  # lower bound
    return (upper >= K >= lower)
    

all_data = all_data.groupby(['expiration_date','K','option_type']).filter(filter1).groupby(['expiration_date','K','option_type']).progress_apply(grouper1)

 99%|█████████▊| 9866/9998 [00:05<00:00, 1447.98it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/.venv/lib/python3.13/site-packages/tqdm/std.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return getattr(df, df_function)(wrapper, **kwargs)
100%|██████████| 9998/9998 [00:06<00:00, 1455.56it/s]


To accurately do the Greek hedging, you need to have the values for all greeks for each outstanding option at each time point.

Basically we need a big Tensor of shape  $ G \in\mathbb{R}^{n\times m\times d}$ with $n$ = timesteps (trading days), $m$ = outstanding options (strike prices and expiration dates), $d$ the greeks (delta,gamma,vega,theta,rho)

#### Small Cells

In [6]:
all_data = all_data.reset_index(drop=True).rename(columns={'index': 'ts_event'})

In [7]:
all_data['r'] = all_data['r'].ffill()
all_data['T'] = (all_data['expiration_date'] - all_data['ts_event']).dt.days / 365.0

In [8]:
underlying['ts_event'] = pd.to_datetime(underlying['ts_event'])
underlying['expiration_date'] = pd.to_datetime(underlying['expiration_date'])

In [9]:
all_data = pd.merge(all_data, 
                    underlying[['ts_event', 'expiration_date', 'futures_close']], 
                    on=['ts_event', 'expiration_date'], 
                    how='left')
all_data['S'] = all_data['S'].fillna(all_data['futures_close'])
all_data = all_data.drop(columns=['futures_close'])


In [10]:
all_data = all_data[ all_data['ts_event'].dt.dayofweek < 5 ]

In [11]:
def grouper2(group):
    group['S'] = group['S'].interpolate(method='spline', order=1)
    group['Value'] = group['Value'].interpolate(method='spline', order=1)
    return group

all_data = all_data.groupby(['expiration_date', 'K', 'option_type']).progress_apply(grouper2).reset_index(drop=True)


 24%|██▍       | 2437/9998 [00:05<00:09, 764.50it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/.venv/lib/python3.13/site-packages/pandas/core/missing.py:604: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  terp = interpolate.UnivariateSpline(x, y, k=order, **kwargs)
100%|█████████▉| 9970/9998 [00:22<00:00, 592.67it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/.venv/lib/python3.13/site-packages/tqdm/std.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns afte

#### Splitting to Calls

In [12]:
calls_data = all_data[all_data['option_type'] == 'C']

In [13]:
calls_data['IV'] = calls_data.progress_apply(lambda row: implied_vol_b76(row['S'], row['K'], row['T'], row['r'], row['Value'], row['option_type']), axis=1)

  0%|          | 1/681332 [00:00<90:29:50,  2.09it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/notebooks/../utils/implied_volatility.py:29: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
100%|██████████| 681332/681332 [06:48<00:00, 1666.97it/s]
/var/folders/7k/ytnljb093xg6bbfksvnhyg940000gn/T/ipykernel_55869/2625644187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_data['IV'] = calls_data.progress_apply(lambda row: implied_vol_b76(row['S'], row['K'], row['T'], row['r'], row['Value'], row['option_type']), axis=1)


In [14]:
def grouper3(group):
    vol_surface = group.pivot(index='K', columns='T', values='IV')
    vol_surface = vol_surface.sort_index().sort_index(axis=1)

    # Interpolate along strikes and maturities
    vol_surface = vol_surface.interpolate(axis=0, method='linear')
    vol_surface = vol_surface.interpolate(axis=1, method='linear')

    # Edge-fill any remaining NaNs
    vol_surface = vol_surface.ffill(axis=0).bfill(axis=0)
    vol_surface = vol_surface.ffill(axis=1).bfill(axis=1)

    filled_df = vol_surface.stack().rename('IV').reset_index()

    # Map interpolated values back into original DataFrame
    iv_map = filled_df.set_index(['K', 'T'])['IV']
    group['IV'] = group.set_index(['K', 'T'])['IV'].fillna(iv_map).values
    return group


In [15]:
calls_data = calls_data.groupby(['ts_event']).progress_apply(grouper3)

100%|█████████▉| 1850/1859 [00:05<00:00, 305.45it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/.venv/lib/python3.13/site-packages/tqdm/std.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return getattr(df, df_function)(wrapper, **kwargs)
100%|██████████| 1859/1859 [00:06<00:00, 301.38it/s]


#### Add Greeks

In [16]:
calls_data['delta'] = calls_data.progress_apply(lambda row: delta(row['S'], row['K'], row['T'], row['r'], row['IV']), axis=1)
calls_data['gamma'] = calls_data.progress_apply(lambda row: gamma(row['S'], row['K'], row['T'], row['r'], row['IV']), axis=1)
calls_data['vega'] = calls_data.progress_apply(lambda row: vega(row['S'], row['K'], row['T'], row['r'], row['IV']), axis=1)


  1%|          | 6016/681332 [00:00<00:57, 11754.63it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/notebooks/../utils/greeks.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  return norm.cdf((np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T)))
  1%|          | 5947/681332 [00:00<00:54, 12311.30it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/notebooks/../utils/greeks.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
/Users/carloteufel/Documents/Projects/risk-neutral-hedging/notebooks/../utils/greeks.py:10: RuntimeWarning: invalid value encountered in scalar divide
  return norm.pdf(
  1%|          | 6008/681332 [00:00<01:02, 10748.80it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/notebooks/../utils/greeks.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  * norm.pdf((np.log(S / K) + (r + 0.5 * sigma**2) * T

In [17]:
calls_data = calls_data.reset_index(drop=True)
calls_data = calls_data[calls_data['T'] != 0]


In [18]:
calls_data.isna().sum()

ts_event            0
Value              12
option_type         0
K                   0
expiration_date     0
S                   0
r                   0
T                   0
IV                  0
delta               0
gamma               0
vega                0
dtype: int64

In [19]:
import pandas as pd 
from pandas.tseries.offsets import DateOffset


def grouper4(group):
    pivot = (
        group
        .set_index(['expiration_date', 'K'])[['delta','gamma','vega']]   # index by the two “option” keys, keep only the Greeks
        .T                                                               # transpose so that the Greeks become the rows
        )
    
    values = pivot.values
    return pd.Series(data = np.linalg.pinv(values) @ np.array([-1, 0, 0]).T, index=pivot.columns)
        



In [20]:
distribution = calls_data.groupby(['ts_event']).progress_apply(grouper4)

 93%|█████████▎| 1738/1859 [00:00<00:00, 2222.60it/s]/Users/carloteufel/Documents/Projects/risk-neutral-hedging/.venv/lib/python3.13/site-packages/tqdm/std.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return getattr(df, df_function)(wrapper, **kwargs)
100%|██████████| 1859/1859 [00:01<00:00, 1592.83it/s]


In [23]:
distribution_horizontal = distribution.unstack(level=['expiration_date','K'])


In [32]:
distribution_horizontal = distribution_horizontal.fillna(0)
distribution_horizontal = distribution_horizontal.sort_index(axis=1, level=[0,1])

In [33]:
distribution_horizontal.to_csv('../results/HedgingDistribution.csv')

In [34]:
distribution_horizontal

expiration_date 2018-06-15                                                    \
K                  2600.0    2630.0    2640.0    2650.0    2655.0    2660.0    
ts_event                                                                       
2018-05-14       -0.150682  0.000000 -0.137838 -0.115566 -0.108156 -0.108949   
2018-05-15       -0.181810 -0.135705 -0.121371 -0.107330 -0.099208 -0.091203   
2018-05-16       -0.132818 -0.122401 -0.085768 -0.062947 -0.085095 -0.065710   
2018-05-17       -0.128766 -0.103386 -0.079972 -0.070919 -0.069636 -0.068280   
2018-05-18       -0.115955 -0.093406 -0.083000 -0.075690 -0.073133 -0.070760   
...                    ...       ...       ...       ...       ...       ...   
2025-06-20        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2025-06-23        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2025-06-24        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2025-06-25        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2025-06-26        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

expiration_date                                          ... 2026-12-18  \
K                 2665.0    2670.0    2675.0    2680.0   ...    7000.0    
ts_event                                                 ...              
2018-05-14      -0.098853 -0.096837 -0.090035 -0.081105  ...        0.0   
2018-05-15      -0.083398 -0.075017 -0.064970 -0.058336  ...        0.0   
2018-05-16      -0.057636 -0.049709 -0.044968 -0.040669  ...        0.0   
2018-05-17      -0.062860 -0.058603 -0.053024 -0.047144  ...        0.0   
2018-05-18      -0.065442 -0.060690 -0.055094 -0.049458  ...        0.0   
...                   ...       ...       ...       ...  ...        ...   
2025-06-20       0.000000  0.000000  0.000000  0.000000  ...        0.0   
2025-06-23       0.000000  0.000000  0.000000  0.000000  ...        0.0   
2025-06-24       0.000000  0.000000  0.000000  0.000000  ...        0.0   
2025-06-25       0.000000  0.000000  0.000000  0.000000  ...        0.0   
2025-06-26       0.000000  0.000000  0.000000  0.000000  ...        0.0   

expiration_date                                             2027-12-17  \
K               7400.0    7500.0  7800.0  8000.0    8500.0     7000.0    
ts_event                                                                 
2018-05-14          0.0  0.000000     0.0     0.0  0.000000        0.0   
2018-05-15          0.0  0.000000     0.0     0.0  0.000000        0.0   
2018-05-16          0.0  0.000000     0.0     0.0  0.000000        0.0   
2018-05-17          0.0  0.000000     0.0     0.0  0.000000        0.0   
2018-05-18          0.0  0.000000     0.0     0.0  0.000000        0.0   
...                 ...       ...     ...     ...       ...        ...   
2025-06-20          0.0  0.052356     0.0     0.0  0.028661        0.0   
2025-06-23          0.0  0.000000     0.0     0.0  0.021630        0.0   
2025-06-24          0.0  0.000000     0.0     0.0  0.000000        0.0   
2025-06-25          0.0  0.000000     0.0     0.0  0.000000        0.0   
2025-06-26          0.0  0.000000     0.0     0.0  0.000000        0.0   

expiration_date                   2028-12-15  
K               7500.0    8000.0     10000.0  
ts_event                                      
2018-05-14          0.0  0.000000        0.0  
2018-05-15          0.0  0.000000        0.0  
2018-05-16          0.0  0.000000        0.0  
2018-05-17          0.0  0.000000        0.0  
2018-05-18          0.0  0.000000        0.0  
...                 ...       ...        ...  
2025-06-20          0.0  0.073727        0.0  
2025-06-23          0.0  0.056230        0.0  
2025-06-24          0.0  0.000000        0.0  
2025-06-25          0.0  0.000000        0.0  
2025-06-26          0.0  0.000000        0.0  

[1859 rows x 4914 columns]

## Rollout 

In [ ]:
data = pd.read_csv('../results/HedgingDistribution.csv', index_col=0, parse_dates=True)